In [57]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import sys
sys.path.append('../')

from config import remote_db_endpoint, remote_db_port
from config import remote_gwsis_dbname, remote_gwsis_dbuser, remote_gwsis_dbpwd
from config import local_gwsis_dbname, local_gwsis_dbuser, local_gwsis_dbpwd 

In [58]:
# Read CSV File into workbook
csv_file = "Resources/craigslist.csv"
craigslist_df = pd.read_csv(csv_file)
craigslist_df.head()

,Item,Location,Price
0,"24"" ASUS LED Computer Monitor",Hill East,120
1,Ultra Wide IPS LED Computer Monitor,Adams Morgan,250
2,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,79
3,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,79
4,Computer Monitor Stand or TV Stand or Printer ...,Laurel,25


In [59]:
#Rename Columns
monitor_df = craigslist_df.rename(columns={"Item":"Model", "Price": "Price($)"})
monitor_df.head()

,Model,Location,Price($)
0,"24"" ASUS LED Computer Monitor",Hill East,120
1,Ultra Wide IPS LED Computer Monitor,Adams Morgan,250
2,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,79
3,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,79
4,Computer Monitor Stand or TV Stand or Printer ...,Laurel,25


In [60]:
# Change values in 'Price($)' columns to currency
monitor_df['Price($)'] = monitor_df['Price($)'].map("${:,.0f}".format)
monitor_df.head()

,Model,Location,Price($)
0,"24"" ASUS LED Computer Monitor",Hill East,$120
1,Ultra Wide IPS LED Computer Monitor,Adams Morgan,$250
2,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,$79
3,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,$79
4,Computer Monitor Stand or TV Stand or Printer ...,Laurel,$25


In [69]:
# Create local database connection
rds_connection_string = "root:Daytona08!@localhost:3306/etl_project"
engine = create_engine(f'mysql://{rds_connection_string}')

In [72]:
# Check for tables
engine.table_names()

['craigslist_data']

In [71]:
# Push dataframe to loacal database
monitor_df.to_sql(name='craigslist_data', con=engine, if_exists='append', index=False)

In [74]:
# Call data from local database
pd.read_sql_query('select * from craigslist_data', con=engine).head(10)

,Model,Location,Price($)
0,"24"" ASUS LED Computer Monitor",Hill East,$120
1,Ultra Wide IPS LED Computer Monitor,Adams Morgan,$250
2,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,$79
3,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,$79
4,Computer Monitor Stand or TV Stand or Printer ...,Laurel,$25
5,"DELL 15"" Flat Panel LCD Display, Computer Moni...",Laurel,$25
6,"DELL 19"" Flat Screen Computer Monitor Display ...",Laurel,$50
7,"GATEWAY 15"" Flat Panel Computer Monitor Displa...",Laurel,$40
8,"DELL 17"" Flat Panel LCD Display, Computer Moni...",Laurel,$40
9,Tyris 19 computer monitor,"Baileys Crossroads, VA",$10


In [80]:
# AWS Database Connection
awsengine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/etl_project")

# Create a remote database engine connection
awsconn = awsengine.connect()

# Write to AWS

monitor_df.to_sql(name='craigslist_data', con=awsconn, if_exists='append', index=False)

In [87]:
# Call data from AWS Database
aws_craigslist_data = pd.read_sql("SELECT * FROM craigslist_data", awsconn)

aws_craigslist_data.head()

,Model,Location,Price($)
0,"24"" ASUS LED Computer Monitor",Hill East,$120
1,Ultra Wide IPS LED Computer Monitor,Adams Morgan,$250
2,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,$79
3,"HP Compaq L1945w 19"" LCD Computer Monitor VGA ...",Potomac,$79
4,Computer Monitor Stand or TV Stand or Printer ...,Laurel,$25
